In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

In [5]:
train_url =  'https://storage.googleapis.com/tf-datasets/titanic/train.csv'
test_url = 'https://storage.googleapis.com/tf-datasets/titanic/eval.csv'

In [6]:
train_path = tf.keras.utils.get_file("train.csv" , train_url)
test_path = tf.keras.utils.get_file("test.csv",test_url)

16384/13049 [=====================================] - 0s 1us/step


In [8]:
df_train = pd.read_csv(train_path)
df_test = pd.read_csv(test_path)

In [11]:
y_train = df_train.pop('survived')
y_test = df_test.pop('survived')

In [12]:
df_train.head()

,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,female,35.0,1,0,53.1000,First,C,Southampton,n
4,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [13]:
Categorical_columns = ['sex', 'class', 'deck','embark_town','alone']
Numeric_columns = ['age','fare']

In [15]:
feature_columns = []
for i in Categorical_columns:
    vocabulary = df_train[i].unique()
    feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(i,vocabulary))

for i in Numeric_columns:
    feature_columns.append(tf.feature_column.numeric_column(i , dtype = tf.float32))


In [22]:
def input_fn(data , labels, epochs = 10 , shuffle = True , batch_size = 32 ):
    ds = tf.data.Dataset.from_tensor_slices((dict(data) , labels))
    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.batch(batch_size).repeat(epochs)
    return ds

In [24]:
estimator = tf.estimator.LinearClassifier(feature_columns = feature_columns)


INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'C:\\Users\\mhatr\\AppData\\Local\\Temp\\tmp89h_bq14', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_checkpoint_save_graph_def': True, '_service': None, '_cluster_spec': ClusterSpec({}), '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [25]:
estimator.train(input_fn = lambda:input_fn(df_train , y_train))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 0...
INFO:tensorflow:Saving checkpoints for 0 into C:\Users\mhatr\AppData\Local\Temp\tmp89h_bq14\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 0...
INFO:tensorflow:loss = 0.6931472, step = 0
INFO:tensorflow:global_step/sec: 282.477
INFO:tensorflow:loss = 0.4738052, step = 100 (0.359 sec)
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 200...
INFO:tensorflow:Saving checkpoints for 200 into C:\Users\mhatr\AppData\Local\Temp\tmp89h_bq14\model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 200...
INFO:tensorflow:Loss for final step: 0.29221192.


In [30]:
result = estimator.evaluate(input_fn = lambda:input_fn(df_test , y_test,epochs = 1, shuffle = False))

INFO:tensorflow:Calling model_fn.


C:\Users\mhatr\anaconda3\lib\site-packages\tensorflow\python\keras\engine\base_layer_v1.py:1727: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2021-05-13T10:12:21Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from C:\Users\mhatr\AppData\Local\Temp\tmp89h_bq14\model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Inference Time : 1.32623s
INFO:tensorflow:Finished evaluation at 2021-05-13-10:12:22
INFO:tensorflow:Saving dict for global step 200: accuracy = 0.7462121, accuracy_baseline = 0.625, auc = 0.8258035, auc_precision_recall = 0.8032781, average_loss = 0.49376845, global_step = 200, label/mean = 0.375, loss = 0.49099275, precision = 0.6632653, prediction/mean = 0.36597177, recall = 0.65656567
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 200: C:\Users\mhatr\AppData\Local\Temp\tmp89h_bq14\model.ckpt-200


In [31]:
result['accuracy']

0.7462121

In [32]:
y_test

0      0
1      0
2      1
3      1
4      1
      ..
259    1
260    0
261    0
262    0
263    1
Name: survived, Length: 264, dtype: int64